In [1]:
%%capture
!pip install --quiet evaluate
!pip install --quiet googletrans==3.1.0a0
!pip install --quiet pyvi
!pip install -U sentence-transformers

In [2]:
from googletrans import Translator
import pandas as pd
import torch
from datasets import load_dataset 
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification , AutoTokenizer , AutoModel
import torch.nn as nn
from tqdm.auto import tqdm
from transformers import DataCollatorWithPadding , get_scheduler
from torch.utils.data import DataLoader
import evaluate
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit , train_test_split
import numpy as np
from accelerate import Accelerator
from pyvi.ViTokenizer import tokenize
device = torch.device ("cuda") if torch.cuda.is_available() else torch.device ("cpu" )
# vietnamese_text = "Xin chào, bạn đang làm gì?"
# english_translation = translate_vietnamese_to_english(vietnamese_text)
# print(english_translation)


2024-03-07 02:20:30.388903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 02:20:30.389018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 02:20:30.509107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

In [4]:
vi_model =  SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
# vi_model =  AutoModel.from_pretrained('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base').to(device)

# eng_model = SentenceTransformer ('princeton-nlp/sup-simcse-roberta-base')
eng_model = SentenceTransformer ('distilbert/distilbert-base-uncased-finetuned-sst-2-english')
# eng_model = AutoModel.from_pretrained ('sentence-transformers/all-mpnet-base-v2')

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
vi_tok = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
# eng_tok = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [6]:
en_vi = load_dataset("opus100", language_pair = "en-vi")
train_data = en_vi["train"].select (range (250000))
train_data

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset opus100 downloaded and prepared to /root/.cache/huggingface/datasets/opus100/default-language_pair=en-vi/1.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['translation'],
    num_rows: 250000
})

In [7]:
train_data[0]

{'translation': {'en': 'What is it?', 'vi': 'Cái gì đó?'}}

In [8]:
#Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [9]:
# def tokenize_vi (samples) :
#     vi_sen = [tokenize(sentence['vi']) for sentence in samples['translation']]
# #     return vi_tok(vi_tokenize, padding = "max_length", truncation= True , max_length = 100) 
#     eng_sen = [sample['en']  for sample in samples["translation"]]
#     print (eng_sen)
#     1/0
#     with torch.no_grad () :
#         vi_embedding_list = vi_model.encode (vi_sen)
#         en_embedding_list = eng_model.encode (eng_sen)
#     return {"en_emb" : en_embedding_list , "vi_emb" : vi_embedding_list }

def embedding_language (samples) :
    vi_tokenize = [tokenize(sentence['vi']) for sentence in samples['translation']]
    eng_sen = [sample['en']  for sample in samples["translation"]]
    with torch.no_grad () :
        vi_emb = vi_model.encode ( vi_tokenize)
        eng_emb = eng_model.encode(eng_sen)
    return {"vi_emb" : vi_emb ,"en_emb" : eng_emb   }


# def embedding_language (samples) :
#     vi_tokenize = [tokenize(sentence['vi']) for sentence in samples['translation']]
    
#     vi_tokenize = vi_tok(vi_tokenize, padding = "max_length", truncation= True , max_length = 100 , return_tensors = "pt").to(device)
    
#     eng_sen = [sample['en']  for sample in samples["translation"]]
#     with torch.no_grad () :
#         vi_emb = vi_model(** vi_tokenize).pooler_output.cpu().numpy().tolist()
        
#         eng_emb = eng_model.encode(eng_sen)
#     return {"vi_emb" : vi_emb ,"en_emb" : eng_emb   }


In [ ]:
%%capture
vi_tok_data = train_data.map (embedding_language , batched = True  )

In [ ]:
len ( vi_tok_data[0]['vi_emb'] )

In [ ]:
vi_tok_data

In [ ]:
data_final = vi_tok_data.remove_columns (['translation'])

In [ ]:
split_dataset = data_final.train_test_split (train_size = 0.8 ,seed = 42 )
split_dataset

In [ ]:
class Adapter (nn.Module) :
    def __init__(self, ):
        super(Adapter, self).__init__()
        self.linear1 = nn.Linear (768 , 384)
        self.linear2 = nn.Linear (384 , 768)
        self.layernorm = nn.LayerNorm (384)
        self.dropout = nn.Dropout (0.1)
        
    
    def forward(self, vi_emb_stack):
        output = self.linear1 (vi_emb_stack)
        output = self.layernorm (output)
        output = self.dropout (output)
        output = self.linear2 (output)
        return output
adapter = Adapter()

In [ ]:
epochs = 5
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader (split_dataset["train"], batch_size= 128 , shuffle= True )
val_dataloader = DataLoader (split_dataset['test'], batch_size= 128 , shuffle= True  )
accelerator = Accelerator ()
device = torch.device ("cuda") if torch.cuda.is_available() else torch.device ("cpu" )

step_to_train = epochs * len (val_dataloader)
optimizer = torch.optim.AdamW (adapter.parameters () , lr = 2e-5)
scheduler = get_scheduler (name = "cosine" , 
                           optimizer = optimizer ,
                           num_warmup_steps=0,
                           num_training_steps = step_to_train
                          )

train_dataloader , val_dataloader , adapter , optimizer= accelerator.prepare (train_dataloader , val_dataloader , adapter , optimizer)


In [ ]:
import numpy as np 
prediction = []
criterion = nn.MSELoss()
best_val_loss = 0
version = 0
for epoch in range (epochs) : 
    train_losses = []
    vi_model.train()
    for batch in tqdm ( train_dataloader ) :
        vi_emb = batch.pop ("vi_emb")
        en_emb = batch.pop ("en_emb")
        
        vi_emb_stack =  torch.stack (vi_emb , dim = 1).float () .to(device)
        en_emb_stack =  torch.stack (en_emb , dim = 1).float () .to(device)
        

        fake_en_emb = adapter.forward (vi_emb_stack)
        loss =  criterion (fake_en_emb , en_emb_stack)

        # BACKWARD ADAPTER
        accelerator.backward (loss)

        optimizer.step ()
        scheduler.step ()
        
        
        optimizer.zero_grad ()
        
        train_losses.append (loss.item())
    
            #print (outputs.shape , labels.shape)
    print ("TRAINING : Epoch : {}, TRAINING Loss: {}".format (epoch + 1, np.mean( train_losses) ) )
    
    vi_model.eval()
    val_losses = []
    for batch in  tqdm (val_dataloader)  :
        
        vi_emb = batch.pop ("vi_emb")
        en_emb = batch.pop ("en_emb")
        
        vi_emb_stack =  torch.stack (vi_emb , dim = 1).float () .to(device)
        en_emb_stack =  torch.stack (en_emb , dim = 1).float () .to(device)
        

        fake_en_emb = adapter.forward (vi_emb_stack)
        loss =  criterion (fake_en_emb , en_emb_stack)
        val_losses.append (loss.item())
    if np.mean( val_losses) > best_val_loss :
        torch.save(vi_model.state_dict(), "vietnam model {}".format (version))
        version +=1
        best_val_loss = np.mean( val_losses)
    print ("VALIDATION : Epoch : {}, Val Loss: {} ".format (epoch + 1, np.mean( val_losses) ))


# TEST ON UIT STUDENT FEEDBACK

In [ ]:
dataset_link = "uitnlp/vietnamese_students_feedback"
from datasets import load_dataset
dataset = load_dataset(dataset_link)
val_data = dataset["validation"]

# PREPROCESS

In [ ]:
def del_neutral (samples) :
    return samples['sentiment'] == 0 or samples['sentiment'] == 2
no_neutral_data = val_data.filter (del_neutral)
no_neutral_data

In [ ]:
num2label = {2 : 1 , 0 : 0 }
# data có label là 2 tương ứng positive còn 0 ứng với negative
# còn của model là {0: 'NEGATIVE', 1: 'POSITIVE'}
def convertlabel (samples) :
    return {"sentiment" : [num2label [num] for num in samples["sentiment"]] }
con_dataset = no_neutral_data.map (convertlabel , batched = True)
con_dataset

In [ ]:
con_dataset

In [ ]:
backbone_vi_model =  SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
# backbone_vi_model =  AutoModel.from_pretrained('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base').to(device)
# vi_tok = AutoTokenizer.from_pretrained ("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

In [ ]:
en_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english").to(device)
en_tok = AutoTokenizer.from_pretrained ("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# NHỚ PHẢI CHẠY FORWARD ADAPTER SAU KHI TRAIN LẠI ADAPTER MỚI NHAN TRỜI

In [ ]:
def forward_adapter (samples) :
    vi_tokenize = [tokenize(sentence) for sentence in samples['sentence']]
    with torch.no_grad () :
        vi_embedding_list = backbone_vi_model.encode (vi_tokenize)
        tensor_samples =  torch.tensor ( vi_embedding_list).to(device)
        fake_en_emb =  [ adapter (tensor_sample) for tensor_sample in tensor_samples ] 
    return {"en_emb" :  fake_en_emb}

In [ ]:
# def forward_adapter_fake (samples) :
#     vi_tokenize = [tokenize(sentence) for sentence in samples['sentence']]
    
#     vi_tokenize = vi_tok(vi_tokenize, padding = "max_length", truncation= True , max_length = 100 , return_tensors = "pt").to(device)
    
#     with torch.no_grad () :
#         vi_emb = vi_model(** vi_tokenize).pooler_output
#         fake_en_emb =  adapter (vi_emb).cpu().numpy().tolist()
#     return {"en_emb" : fake_en_emb   }

In [ ]:
eval_dataset = con_dataset.map (forward_adapter , batched = True , remove_columns = ["sentence" , "topic" ])

In [ ]:
# def forward_adapter (samples) :
#     tensor_samples = torch.tensor ( samples["vi_emb"] ).to(device)
#     fake_en_emb =  [ adapter (tensor_sample) for tensor_sample in tensor_samples ] 
#     return {"en_emb" :  fake_en_emb}

In [ ]:
# en_emb_data = eval_dataset.map (forward_adapter , batched = True , remove_columns = ["sentence" , "topic" , "vi_emb"])

In [ ]:

val_dataloader = DataLoader (eval_dataset, batch_size= 128, shuffle=True  )
#Metric
accelerator = Accelerator ()
val_dataloader ,adapter= accelerator.prepare ( val_dataloader , adapter )


metric_valid_acc = evaluate.load("accuracy")

metric_valid_f1 = evaluate.load("f1")

In [ ]:
# def vie_emb (samples) :
#     vi_tokenize = [tokenize(sentence) for sentence in samples['sentence']]
#     token = vi_tok(vi_tokenize, padding = "max_length", truncation= True , max_length = 100 , return_tensors = "pt") 
#     vi_emb = vi_model (**token)
#     mean_pool_vi = mean_pooling (vi_emb , token["attention_mask"])
#     vi_embedding_list = mean_pool_vi.numpy().tolist ()
#     return {"vi_emb" :vi_embedding_list }
#             #, "en" : [eng_model ( sample['en'] ) for sample in samples["translation"]] 

In [ ]:
# input_dataset = con_dataset.map (vie_emb , batched = True , remove_columns = ['sentence', 'topic'])
# input_dataset

In [ ]:
# def forward_adapter (samples) :
#     tensor_samples = torch.tensor ( samples["vi_emb"] ).to(device)
#     fake_en_emb =  [ adapter (tensor_sample) for tensor_sample in tensor_samples ] 
#     return {"en_emb" :  fake_en_emb}

In [ ]:
adapter.eval()
val_losses = []
prediction = []
criterion = nn.CrossEntropyLoss()
for batch in tqdm ( val_dataloader ) :
    with torch.no_grad () :
        labels = batch.pop ("sentiment").to(device)
        en_emb = batch.pop ("en_emb")
        en_emb_stack = torch.stack (en_emb , dim = 1 ).float ().to(device)
        with torch.no_grad () :
            output = en_model.pre_classifier.forward (en_emb_stack)
            output = en_model.classifier.forward (output)
            logits = en_model.dropout.forward(output)
        
        #print (outputs.shape , labels.shape)
        loss = criterion(logits,labels)
        predict = torch.argmax (logits, dim = -1)
        
        
        val_losses.append (loss.item())
        prediction.append (predict)
        metric_valid_acc.add_batch (references=labels, predictions = predict)
        metric_valid_f1.add_batch (references=labels, predictions = predict )

val_acc = metric_valid_acc.compute ()['accuracy']
# if val_acc > best_val_acc :
#     torch.save(model.state_dict(), "Phobert_v2_best_model_full_data_{}".format (i))
#     best_val_acc = val_acc
#     i+=1
print (" Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format ( np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="macro")['f1'] ))

In [ ]:
encoding

In [ ]:
model

In [ ]:
model.vilt